# Task A. MongoDB Data Model
```
Chosen Data Model:

{
    "_id": datetime.datetime(2017, 12, 27, 0, 0),
    "station": 948702,
    "air_temperatue_celsius": 18,
    "relative_humidity": 58.3,
    "windspeed_knots": 9.2,
    "max_wind_speed": 15.9,
    "precipitation": ' 0.00I',
    "hotspots": [
                    {
                    "latitude": -37.0193,
                    "longitude": 148.1459,
                    "datetime": datetime.datetime(2017, 3, 6, 5, 6, 20),
                    "confidence": 84,
                    "surface_temperature_celsius: 42
                    }
                ]
}
```

The climate is based on a consecutive and unique dates. While there may be some hotspots on a particular day in different time period, there may be no hotspot on a particular day as well. Thus, the two datasets has a one to many relationship in which a date can have one or many hotspots. As a result, we will want a data model that can view the hotspots data based on specific dates. We have decided on a schema which will embed the hotspot data in the each date together with the climate data, in which the hotspot data are being kept in an array. Doing this will prevent us from repeating the climates data as we know each date can have one or many hotspots, thus, reducing the space required for storing this data model. Furthermore, with this embedded data model, we can retrieve the hotspots information on a specific date with one query. Note that we are using the `date` as the id of our data model as it is unique in the climate data.

Even though embedded data model is not recommended if the embedded fields (i.e. hotspot) are large since it might exceed the space limitation, we know from the dataset that number of hotspots on one specific date is not a large number. Hence, an embedded data model will still be suitable in this case.

# Task B. Querying MongoDB using PyMongo

In [1]:
import pymongo # for running mongo DB in python
from pymongo import MongoClient # to connect to mongod
from datetime import datetime # to convert data types to datetime format
from pprint import pprint # for pretty print results in pymongo

## Task B.1
We are first required to write a python program that will read the data from `hotspot_historic.csv` and `climate_historic.csv` and load them to the new database `fit5148_assignment_db`. In this database, we will create a collection based on the data model we have chosen in Task A.

First, we make a connection to `mongod` and afterwhich, we create our `fit5148_assignment_db` database. Then we will create a collection `climates` in this database which is based on the data model we have defined in Task A.

In [2]:
# connect on the default host and port
client = MongoClient ()

# use the fit5148_assignment_db which has yet to be created
db = client.fit5148_assignment_db

# create one collection, climates
climates = db.climates

We load in `climate_historic.csv` data and store them in a list `climate_raw`.<br>
Note that the first line contains the column headers.

In [3]:
# load in climate_historic.csv and store in climate_raw
with open("./climate_historic.csv") as file:
    climate_raw = file.readlines()
    # strip the new line for each line
    climate_raw = [line.strip() for line in climate_raw]

climate_raw

['station,date,air_temperature_celcius,relative_humidity,windspeed_knots,max_wind_speed,precipitation',
 '948700,31/12/2016,19,56.8,7.9,11.1, 0.00I',
 '948700,2/01/2017,15,50.7,9.2,13, 0.02G',
 '948700,3/01/2017,16,53.6,8.1,15, 0.00G',
 '948700,4/01/2017,24,61.6,7.7,14, 0.00I',
 '948700,5/01/2017,24,62.3,7,13, 0.00I',
 '948700,6/01/2017,26,60.8,6.9,12, 0.00I',
 '948700,7/01/2017,32,54.1,12.8,19, 0.00I',
 '948700,8/01/2017,20,57.7,9.9,18.1, 0.00I',
 '948700,9/01/2017,19,56.3,5.8,8.9, 0.00I',
 '948700,10/01/2017,20,57,8.7,13, 0.00I',
 '948700,11/01/2017,18,53.3,7.9,14, 0.00I',
 '948700,12/01/2017,22,56.3,8,13, 0.08G',
 '948700,13/01/2017,19,54.1,11.2,18.1, 0.31G',
 '948700,14/01/2017,17,50.4,11.9,16.9, 0.00G',
 '948700,15/01/2017,16,50.8,5.8,12, 0.00I',
 '948700,16/01/2017,24,55.4,5.9,9.9, 0.00I',
 '948700,17/01/2017,24,53.4,9.9,15.9, 0.00I',
 '948700,18/01/2017,16,46.7,10,16.9, 0.00I',
 '948700,19/01/2017,19,57.2,8.9,15, 0.98G',
 '948700,20/01/2017,16,50.9,12.9,21, 0.03G',
 '948700,21/0

We then write a function that will create our `climates` collection with the `climate_raw` data first.

In this function, we will loop through line 2 to last line without the first line as the first line contains the column headers, and create a record for each line with all the attributes. We will set the `_id` of our `hotspots` collection using the `date` column in `climate_raw` data. Each attribute is then stored in the record with the chosen data type as mentioned in Task A. Afterwhich, we apply this function with our `climates` collection defined earlier which will then insert all the records of `climate_raw` data into the collection.

Note that `date` is in the format of `'%d/%m/%Y'`, in which the documentation of the representation can be found <a href="https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behaviour">here</a>.

In [4]:
def create_climate(collection):
    # read each line from line 2 to end (ignore 1st line since it contains the headers)
    for i in range(1,len(climate_raw)):
        # for each line, we split the line with ',' and store them in a list
        line = climate_raw[i].split(',')
        # create a record for each line in climate_raw data
        climate_record = {
            # set date as the _id in record, convert it to datetime format
            '_id':datetime.strptime(line[1],'%d/%m/%Y'),
            # set the rest of the records for the rest of the variables
            'station':int(line[0]),
            'air_temperature_celsius':int(line[2]),
            'relative_humidity':float(line[3]),
            'windspeed_knots':float(line[4]),
            'max_wind_speed':float(line[5]),
            'precipitation':str(line[6]),
            # create an empty array that will store the hotspots data later
            'hotspots':[]
        }
        collection.insert_one(climate_record)

# run this function with climates collection created earlier on
create_climate(climates)      

In [5]:
# pretty print out the current hotspots collection after adding in climate data
for document in climates.find({}):
    pprint(document)

{'_id': datetime.datetime(2016, 12, 31, 0, 0),
 'air_temperature_celsius': 19,
 'hotspots': [],
 'max_wind_speed': 11.1,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.8,
 'station': 948700,
 'windspeed_knots': 7.9}
{'_id': datetime.datetime(2017, 1, 2, 0, 0),
 'air_temperature_celsius': 15,
 'hotspots': [],
 'max_wind_speed': 13.0,
 'precipitation': ' 0.02G',
 'relative_humidity': 50.7,
 'station': 948700,
 'windspeed_knots': 9.2}
{'_id': datetime.datetime(2017, 1, 3, 0, 0),
 'air_temperature_celsius': 16,
 'hotspots': [],
 'max_wind_speed': 15.0,
 'precipitation': ' 0.00G',
 'relative_humidity': 53.6,
 'station': 948700,
 'windspeed_knots': 8.1}
{'_id': datetime.datetime(2017, 1, 4, 0, 0),
 'air_temperature_celsius': 24,
 'hotspots': [],
 'max_wind_speed': 14.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 61.6,
 'station': 948700,
 'windspeed_knots': 7.7}
{'_id': datetime.datetime(2017, 1, 5, 0, 0),
 'air_temperature_celsius': 24,
 'hotspots': [],
 'max_wind_speed': 13.0,

 'station': 948701,
 'windspeed_knots': 12.8}
{'_id': datetime.datetime(2017, 3, 24, 0, 0),
 'air_temperature_celsius': 20,
 'hotspots': [],
 'max_wind_speed': 11.1,
 'precipitation': ' 0.00I',
 'relative_humidity': 58.4,
 'station': 948701,
 'windspeed_knots': 6.1}
{'_id': datetime.datetime(2017, 3, 25, 0, 0),
 'air_temperature_celsius': 23,
 'hotspots': [],
 'max_wind_speed': 15.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 61.9,
 'station': 948701,
 'windspeed_knots': 8.0}
{'_id': datetime.datetime(2017, 3, 26, 0, 0),
 'air_temperature_celsius': 23,
 'hotspots': [],
 'max_wind_speed': 26.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 60.6,
 'station': 948701,
 'windspeed_knots': 10.1}
{'_id': datetime.datetime(2017, 3, 27, 0, 0),
 'air_temperature_celsius': 17,
 'hotspots': [],
 'max_wind_speed': 25.1,
 'precipitation': ' 0.16G',
 'relative_humidity': 53.4,
 'station': 948701,
 'windspeed_knots': 9.4}
{'_id': datetime.datetime(2017, 3, 28, 0, 0),
 'air_temperature_celsi

{'_id': datetime.datetime(2017, 6, 21, 0, 0),
 'air_temperature_celsius': 10,
 'hotspots': [],
 'max_wind_speed': 7.0,
 'precipitation': ' 0.00G',
 'relative_humidity': 45.9,
 'station': 948701,
 'windspeed_knots': 3.2}
{'_id': datetime.datetime(2017, 6, 22, 0, 0),
 'air_temperature_celsius': 10,
 'hotspots': [],
 'max_wind_speed': 14.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 41.4,
 'station': 948701,
 'windspeed_knots': 9.4}
{'_id': datetime.datetime(2017, 6, 23, 0, 0),
 'air_temperature_celsius': 11,
 'hotspots': [],
 'max_wind_speed': 16.9,
 'precipitation': ' 0.12G',
 'relative_humidity': 43.9,
 'station': 948701,
 'windspeed_knots': 11.2}
{'_id': datetime.datetime(2017, 6, 24, 0, 0),
 'air_temperature_celsius': 10,
 'hotspots': [],
 'max_wind_speed': 14.0,
 'precipitation': ' 0.02G',
 'relative_humidity': 37.9,
 'station': 948701,
 'windspeed_knots': 8.5}
{'_id': datetime.datetime(2017, 6, 25, 0, 0),
 'air_temperature_celsius': 11,
 'hotspots': [],
 'max_wind_speed': 15

{'_id': datetime.datetime(2017, 8, 22, 0, 0),
 'air_temperature_celsius': 12,
 'hotspots': [],
 'max_wind_speed': 11.1,
 'precipitation': ' 0.00G',
 'relative_humidity': 43.3,
 'station': 948702,
 'windspeed_knots': 7.0}
{'_id': datetime.datetime(2017, 8, 23, 0, 0),
 'air_temperature_celsius': 10,
 'hotspots': [],
 'max_wind_speed': 12.0,
 'precipitation': ' 0.39G',
 'relative_humidity': 44.1,
 'station': 948702,
 'windspeed_knots': 7.3}
{'_id': datetime.datetime(2017, 8, 24, 0, 0),
 'air_temperature_celsius': 9,
 'hotspots': [],
 'max_wind_speed': 9.9,
 'precipitation': ' 0.01G',
 'relative_humidity': 42.2,
 'station': 948702,
 'windspeed_knots': 6.4}
{'_id': datetime.datetime(2017, 8, 25, 0, 0),
 'air_temperature_celsius': 12,
 'hotspots': [],
 'max_wind_speed': 8.9,
 'precipitation': ' 0.00G',
 'relative_humidity': 40.4,
 'station': 948702,
 'windspeed_knots': 5.9}
{'_id': datetime.datetime(2017, 8, 26, 0, 0),
 'air_temperature_celsius': 11,
 'hotspots': [],
 'max_wind_speed': 19.0,

 'windspeed_knots': 9.6}
{'_id': datetime.datetime(2017, 10, 20, 0, 0),
 'air_temperature_celsius': 13,
 'hotspots': [],
 'max_wind_speed': 16.9,
 'precipitation': ' 0.00G',
 'relative_humidity': 42.0,
 'station': 948702,
 'windspeed_knots': 11.4}
{'_id': datetime.datetime(2017, 10, 21, 0, 0),
 'air_temperature_celsius': 12,
 'hotspots': [],
 'max_wind_speed': 14.0,
 'precipitation': ' 0.02G',
 'relative_humidity': 47.3,
 'station': 948702,
 'windspeed_knots': 9.7}
{'_id': datetime.datetime(2017, 10, 22, 0, 0),
 'air_temperature_celsius': 13,
 'hotspots': [],
 'max_wind_speed': 13.0,
 'precipitation': ' 0.02G',
 'relative_humidity': 50.3,
 'station': 948702,
 'windspeed_knots': 8.6}
{'_id': datetime.datetime(2017, 10, 23, 0, 0),
 'air_temperature_celsius': 14,
 'hotspots': [],
 'max_wind_speed': 12.0,
 'precipitation': ' 0.00G',
 'relative_humidity': 49.5,
 'station': 948702,
 'windspeed_knots': 6.0}
{'_id': datetime.datetime(2017, 10, 24, 0, 0),
 'air_temperature_celsius': 15,
 'hotsp

 'relative_humidity': 58.7,
 'station': 948702,
 'windspeed_knots': 7.8}
{'_id': datetime.datetime(2017, 12, 18, 0, 0),
 'air_temperature_celsius': 25,
 'hotspots': [],
 'max_wind_speed': 8.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 60.9,
 'station': 948702,
 'windspeed_knots': 6.3}
{'_id': datetime.datetime(2017, 12, 19, 0, 0),
 'air_temperature_celsius': 21,
 'hotspots': [],
 'max_wind_speed': 22.9,
 'precipitation': ' 0.67G',
 'relative_humidity': 57.4,
 'station': 948702,
 'windspeed_knots': 10.7}
{'_id': datetime.datetime(2017, 12, 20, 0, 0),
 'air_temperature_celsius': 17,
 'hotspots': [],
 'max_wind_speed': 18.1,
 'precipitation': ' 0.00G',
 'relative_humidity': 51.2,
 'station': 948702,
 'windspeed_knots': 9.9}
{'_id': datetime.datetime(2017, 12, 21, 0, 0),
 'air_temperature_celsius': 17,
 'hotspots': [],
 'max_wind_speed': 11.1,
 'precipitation': ' 0.00I',
 'relative_humidity': 52.5,
 'station': 948702,
 'windspeed_knots': 5.7}
{'_id': datetime.datetime(2017, 12, 22,

Next we will load in the `hotspot_historic.csv` data as well and store them in a list `hotspot_raw`.

In [6]:
# load in hotspot_raw.csv and store in hotspot_raw
with open("./hotspot_historic.csv") as file:
    hotspot_raw = file.readlines()
    # strip the new line for each line
    hotspot_raw = [line.strip() for line in hotspot_raw]
    
hotspot_raw

['latitude,longitude,datetime,confidence,date,surface_temperature_celcius',
 '-37.966,145.051,2017-12-27T04:16:51,78,27/12/2017,68',
 '-35.541,143.311,2017-12-27T00:02:15,82,27/12/2017,63',
 '-35.554,143.307,2017-12-27T00:02:15,67,27/12/2017,53',
 '-35.543,143.316,2017-12-27T00:02:14,86,27/12/2017,67',
 '-37.708,145.1,2017-12-25T04:29:08,80,25/12/2017,54',
 '-35.646,142.282,2017-12-24T13:12:01,65,24/12/2017,32',
 '-38.075,143.76,2017-12-21T04:53:29,72,21/12/2017,46',
 '-37.636,149.33,2017-12-16T15:38:39,94,16/12/2017,43',
 '-37.624,149.332,2017-12-16T15:38:39,69,16/12/2017,33',
 '-37.82,142.323,2017-12-16T04:35:15,62,16/12/2017,54',
 '-37.875,142.51,2017-12-16T04:35:13,93,16/12/2017,73',
 '-37.608,149.328,2017-12-16T04:34:58,84,16/12/2017,55',
 '-37.613,149.305,2017-12-16T04:34:58,95,16/12/2017,75',
 '-37.644,149.346,2017-12-16T04:34:57,59,16/12/2017,55',
 '-37.624,149.314,2017-12-16T04:34:57,90,16/12/2017,66',
 '-37.661,149.327,2017-12-16T04:34:57,75,16/12/2017,56',
 '-37.649,149.322,

Then, we will write a function to update these `hotspot_raw` data into the previously created `climates` collection which currently contains only the `climate_raw` data.

Similarly, we will loop through from line 2 to end of the line in `hotspot_raw` data and create a record for each line with the attributes. We will then update the record into `climates` collection using `update_one`. In `update_one`, we will find the record that matches the `date` and then using `push` to embed a hotspot record. Note that we use `push` instead of `set` since we know one particular date can have many hotspots.

Note that `date` is in the format of `'%d/%m/%Y'`<br>
Note that `datetime` is in the format of `'%Y-%m-%d,%H:%M:%S'` after replacing the `T` with `,`

In [7]:
def update_hotspot(collection):
    # read each line from line 2 to end (ignore 1st line since it contains the headers)
    for i in range(1,len(hotspot_raw)):
        # for each line, we split the line with ',' and store them in a list
        line = hotspot_raw[i].split(',')
        # convert date to datetime format for matching later
        new_date = datetime.strptime(line[4],'%d/%m/%Y')
        # convert datetime to a more manipulable format by removing the `T`
        new_datetime = ','.join(line[2].split('T'))
        # create a record for each line in hotspot_raw data
        hotspot_record = {
            'latitude':float(line[0]),
            'longitude':float(line[1]),
            'datetime':datetime.strptime(new_datetime,'%Y-%m-%d,%H:%M:%S'),
            'confidence':int(line[3]),
            'surface_temperature_celsius':int(line[5]),
        }
        # we will update the record in the hotspots collection based on the '_id' which is the date
        collection.update_one({'_id':new_date},{'$push': {"hotspots":hotspot_record}})
        
# run this function with climates collection created earlier on
update_hotspot(climates)      

In [8]:
# pretty print out the current hotspots collection after adding in hotspots data
for document in climates.find({}):
    pprint(document)

{'_id': datetime.datetime(2016, 12, 31, 0, 0),
 'air_temperature_celsius': 19,
 'hotspots': [],
 'max_wind_speed': 11.1,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.8,
 'station': 948700,
 'windspeed_knots': 7.9}
{'_id': datetime.datetime(2017, 1, 2, 0, 0),
 'air_temperature_celsius': 15,
 'hotspots': [],
 'max_wind_speed': 13.0,
 'precipitation': ' 0.02G',
 'relative_humidity': 50.7,
 'station': 948700,
 'windspeed_knots': 9.2}
{'_id': datetime.datetime(2017, 1, 3, 0, 0),
 'air_temperature_celsius': 16,
 'hotspots': [],
 'max_wind_speed': 15.0,
 'precipitation': ' 0.00G',
 'relative_humidity': 53.6,
 'station': 948700,
 'windspeed_knots': 8.1}
{'_id': datetime.datetime(2017, 1, 4, 0, 0),
 'air_temperature_celsius': 24,
 'hotspots': [],
 'max_wind_speed': 14.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 61.6,
 'station': 948700,
 'windspeed_knots': 7.7}
{'_id': datetime.datetime(2017, 1, 5, 0, 0),
 'air_temperature_celsius': 24,
 'hotspots': [],
 'max_wind_speed': 13.0,

 'hotspots': [],
 'max_wind_speed': 15.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 58.8,
 'station': 948701,
 'windspeed_knots': 11.5}
{'_id': datetime.datetime(2017, 3, 6, 0, 0),
 'air_temperature_celsius': 20,
 'hotspots': [{'confidence': 87,
               'datetime': datetime.datetime(2017, 3, 6, 5, 6, 30),
               'latitude': -34.3795,
               'longitude': 141.6331,
               'surface_temperature_celsius': 62},
              {'confidence': 85,
               'datetime': datetime.datetime(2017, 3, 6, 5, 6, 20),
               'latitude': -34.3735,
               'longitude': 141.6604,
               'surface_temperature_celsius': 59}],
 'max_wind_speed': 21.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 52.9,
 'station': 948701,
 'windspeed_knots': 17.0}
{'_id': datetime.datetime(2017, 3, 7, 0, 0),
 'air_temperature_celsius': 19,
 'hotspots': [{'confidence': 88,
               'datetime': datetime.datetime(2017, 3, 7, 4, 16, 10),
               'la

 'windspeed_knots': 10.8}
{'_id': datetime.datetime(2017, 3, 17, 0, 0),
 'air_temperature_celsius': 18,
 'hotspots': [{'confidence': 76,
               'datetime': datetime.datetime(2017, 3, 17, 4, 48, 20),
               'latitude': -34.3383,
               'longitude': 141.543,
               'surface_temperature_celsius': 56},
              {'confidence': 94,
               'datetime': datetime.datetime(2017, 3, 17, 4, 48, 20),
               'latitude': -36.7152,
               'longitude': 141.3853,
               'surface_temperature_celsius': 74},
              {'confidence': 78,
               'datetime': datetime.datetime(2017, 3, 17, 4, 46, 40),
               'latitude': -34.328,
               'longitude': 141.5402,
               'surface_temperature_celsius': 58},
              {'confidence': 84,
               'datetime': datetime.datetime(2017, 3, 17, 4, 45, 10),
               'latitude': -36.6867,
               'longitude': 141.137,
               'surface_temperatur

               'surface_temperature_celsius': 54},
              {'confidence': 94,
               'datetime': datetime.datetime(2017, 3, 28, 4, 30, 10),
               'latitude': -36.7806,
               'longitude': 141.5597,
               'surface_temperature_celsius': 74},
              {'confidence': 79,
               'datetime': datetime.datetime(2017, 3, 28, 4, 30, 10),
               'latitude': -37.639,
               'longitude': 142.0648,
               'surface_temperature_celsius': 52},
              {'confidence': 72,
               'datetime': datetime.datetime(2017, 3, 28, 4, 30, 10),
               'latitude': -36.7791,
               'longitude': 141.5711,
               'surface_temperature_celsius': 46},
              {'confidence': 98,
               'datetime': datetime.datetime(2017, 3, 28, 4, 30, 10),
               'latitude': -34.3526,
               'longitude': 141.6458,
               'surface_temperature_celsius': 82},
              {'confidence': 81,
 

              {'confidence': 75,
               'datetime': datetime.datetime(2017, 4, 3, 3, 52, 10),
               'latitude': -37.8717,
               'longitude': 143.3652,
               'surface_temperature_celsius': 49},
              {'confidence': 66,
               'datetime': datetime.datetime(2017, 4, 3, 3, 52, 10),
               'latitude': -37.8717,
               'longitude': 143.2566,
               'surface_temperature_celsius': 43},
              {'confidence': 83,
               'datetime': datetime.datetime(2017, 4, 3, 3, 51, 30),
               'latitude': -37.3851,
               'longitude': 145.8878,
               'surface_temperature_celsius': 63},
              {'confidence': 89,
               'datetime': datetime.datetime(2017, 4, 3, 3, 51, 30),
               'latitude': -37.7446,
               'longitude': 143.4085,
               'surface_temperature_celsius': 66},
              {'confidence': 75,
               'datetime': datetime.datetime(2017, 4, 3

...

{'_id': datetime.datetime(2017, 7, 30, 0, 0),
 'air_temperature_celsius': 9,
 'hotspots': [],
 'max_wind_speed': 12.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 42.0,
 'station': 948701,
 'windspeed_knots': 4.6}
{'_id': datetime.datetime(2017, 7, 31, 0, 0),
 'air_temperature_celsius': 7,
 'hotspots': [{'confidence': 100,
               'datetime': datetime.datetime(2017, 7, 31, 13, 21),
               'latitude': -37.8641,
               'longitude': 143.4276,
               'surface_temperature_celsius': 51},
              {'confidence': 92,
               'datetime': datetime.datetime(2017, 7, 31, 13, 21),
               'latitude': -37.8658,
               'longitude': 143.4162,
               'surface_temperature_celsius': 43}],
 'max_wind_speed': 8.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 40.5,
 'station': 948701,
 'windspeed_knots': 3.1}
{'_id': datetime.datetime(2017, 8, 1, 0, 0),
 'air_temperature_celsius': 7,
 'hotspots': [{'confidence': 89,
               

               'surface_temperature_celsius': 60}],
 'max_wind_speed': 18.1,
 'precipitation': ' 0.00G',
 'relative_humidity': 41.7,
 'station': 948702,
 'windspeed_knots': 12.3}
{'_id': datetime.datetime(2017, 9, 21, 0, 0),
 'air_temperature_celsius': 15,
 'hotspots': [{'confidence': 73,
               'datetime': datetime.datetime(2017, 9, 21, 5, 11, 47),
               'latitude': -37.227,
               'longitude': 142.138,
               'surface_temperature_celsius': 39},
              {'confidence': 76,
               'datetime': datetime.datetime(2017, 9, 21, 5, 11, 46),
               'latitude': -37.228,
               'longitude': 142.146,
               'surface_temperature_celsius': 42}],
 'max_wind_speed': 22.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 47.3,
 'station': 948702,
 'windspeed_knots': 8.6}
{'_id': datetime.datetime(2017, 9, 22, 0, 0),
 'air_temperature_celsius': 20,
 'hotspots': [{'confidence': 71,
               'datetime': datetime.datetime(2017, 

               'datetime': datetime.datetime(2017, 10, 3, 3, 58, 14),
               'latitude': -37.469,
               'longitude': 148.103,
               'surface_temperature_celsius': 59},
              {'confidence': 73,
               'datetime': datetime.datetime(2017, 10, 3, 1, 22, 44),
               'latitude': -37.229,
               'longitude': 141.182,
               'surface_temperature_celsius': 39},
              {'confidence': 54,
               'datetime': datetime.datetime(2017, 10, 3, 1, 22, 44),
               'latitude': -37.227,
               'longitude': 141.146,
               'surface_temperature_celsius': 31}],
 'max_wind_speed': 12.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 46.2,
 'station': 948702,
 'windspeed_knots': 5.6}
{'_id': datetime.datetime(2017, 10, 4, 0, 0),
 'air_temperature_celsius': 19,
 'hotspots': [{'confidence': 50,
               'datetime': datetime.datetime(2017, 10, 4, 4, 41, 36),
               'latitude': -36.418,
        

 'precipitation': ' 0.24G',
 'relative_humidity': 44.9,
 'station': 948702,
 'windspeed_knots': 16.1}
{'_id': datetime.datetime(2017, 10, 30, 0, 0),
 'air_temperature_celsius': 11,
 'hotspots': [],
 'max_wind_speed': 14.0,
 'precipitation': ' 0.20G',
 'relative_humidity': 43.2,
 'station': 948702,
 'windspeed_knots': 8.1}
{'_id': datetime.datetime(2017, 10, 31, 0, 0),
 'air_temperature_celsius': 13,
 'hotspots': [],
 'max_wind_speed': 12.0,
 'precipitation': ' 0.01G',
 'relative_humidity': 47.1,
 'station': 948702,
 'windspeed_knots': 8.1}
{'_id': datetime.datetime(2017, 11, 1, 0, 0),
 'air_temperature_celsius': 13,
 'hotspots': [],
 'max_wind_speed': 12.0,
 'precipitation': ' 0.08G',
 'relative_humidity': 50.1,
 'station': 948702,
 'windspeed_knots': 8.5}
{'_id': datetime.datetime(2017, 11, 2, 0, 0),
 'air_temperature_celsius': 12,
 'hotspots': [],
 'max_wind_speed': 14.0,
 'precipitation': ' 0.16G',
 'relative_humidity': 43.8,
 'station': 948702,
 'windspeed_knots': 6.7}
{'_id': date

               'surface_temperature_celsius': 55},
              {'confidence': 69,
               'datetime': datetime.datetime(2017, 11, 30, 0, 20, 53),
               'latitude': -37.6,
               'longitude': 149.297,
               'surface_temperature_celsius': 50},
              {'confidence': 64,
               'datetime': datetime.datetime(2017, 11, 30, 0, 20, 49),
               'latitude': -37.382,
               'longitude': 149.341,
               'surface_temperature_celsius': 41}],
 'max_wind_speed': 15.0,
 'precipitation': ' 0.20G',
 'relative_humidity': 61.0,
 'station': 948702,
 'windspeed_knots': 9.3}
{'_id': datetime.datetime(2017, 12, 1, 0, 0),
 'air_temperature_celsius': 17,
 'hotspots': [],
 'max_wind_speed': 15.0,
 'precipitation': ' 0.75G',
 'relative_humidity': 59.0,
 'station': 948702,
 'windspeed_knots': 9.4}
{'_id': datetime.datetime(2017, 12, 2, 0, 0),
 'air_temperature_celsius': 15,
 'hotspots': [],
 'max_wind_speed': 20.0,
 'precipitation': ' 1.77G',

## Task B.2

In [3]:
# B.2a - find climate data on 10th Dec 2017
climates.find_one({"_id": datetime(2017,12,27,0,0)},{"hotspots":0})

{'_id': datetime.datetime(2017, 12, 27, 0, 0),
 'air_temperature_celsius': 28,
 'max_wind_speed': 15.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 58.3,
 'station': 948702,
 'windspeed_knots': 9.3}

In [10]:
# B.2b - find latitude, longitude, surface_temp, confidence when surface_temp was between 65 and 100
results = climates.aggregate([{"$unwind":"$hotspots"},\
                              {"$match":{"hotspots.surface_temperature_celsius":{"$gt":65,"$lt":100}}},\
                              {"$project":{"_id":0,"hotspots.latitude":1,"hotspots.longitude":1,\
                                           "hotspots.surface_temperature_celsius":1,"hotspots.confidence":1}}])
for result in results:
    pprint(result)

{'hotspots': {'confidence': 94,
              'latitude': -37.2284,
              'longitude': 147.9187,
              'surface_temperature_celsius': 73}}
{'hotspots': {'confidence': 97,
              'latitude': -37.6572,
              'longitude': 142.0703,
              'surface_temperature_celsius': 80}}
{'hotspots': {'confidence': 84,
              'latitude': -37.0193,
              'longitude': 148.1459,
              'surface_temperature_celsius': 71}}
{'hotspots': {'confidence': 100,
              'latitude': -37.4229,
              'longitude': 147.027,
              'surface_temperature_celsius': 99}}
{'hotspots': {'confidence': 80,
              'latitude': -37.0055,
              'longitude': 148.1582,
              'surface_temperature_celsius': 68}}
{'hotspots': {'confidence': 85,
              'latitude': -37.4128,
              'longitude': 147.0242,
              'surface_temperature_celsius': 98}}
{'hotspots': {'confidence': 90,
              'latitude': -34.357,
   

{'hotspots': {'confidence': 91,
              'latitude': -37.3803,
              'longitude': 145.611,
              'surface_temperature_celsius': 68}}
{'hotspots': {'confidence': 97,
              'latitude': -37.7617,
              'longitude': 143.081,
              'surface_temperature_celsius': 81}}
{'hotspots': {'confidence': 92,
              'latitude': -37.8161,
              'longitude': 142.9286,
              'surface_temperature_celsius': 71}}
{'hotspots': {'confidence': 90,
              'latitude': -36.0387,
              'longitude': 145.6787,
              'surface_temperature_celsius': 67}}
{'hotspots': {'confidence': 94,
              'latitude': -36.6349,
              'longitude': 142.2125,
              'surface_temperature_celsius': 86}}
{'hotspots': {'confidence': 86,
              'latitude': -36.6986,
              'longitude': 142.7259,
              'surface_temperature_celsius': 72}}
{'hotspots': {'confidence': 94,
              'latitude': -38.0052,
    

              'longitude': 144.9371,
              'surface_temperature_celsius': 69}}
{'hotspots': {'confidence': 100,
              'latitude': -38.0254,
              'longitude': 142.3959,
              'surface_temperature_celsius': 99}}
{'hotspots': {'confidence': 99,
              'latitude': -37.8885,
              'longitude': 143.9157,
              'surface_temperature_celsius': 85}}
{'hotspots': {'confidence': 95,
              'latitude': -35.0706,
              'longitude': 141.4435,
              'surface_temperature_celsius': 75}}
{'hotspots': {'confidence': 100,
              'latitude': -38.0795,
              'longitude': 143.2449,
              'surface_temperature_celsius': 91}}
{'hotspots': {'confidence': 100,
              'latitude': -35.1857,
              'longitude': 141.0604,
              'surface_temperature_celsius': 92}}
{'hotspots': {'confidence': 100,
              'latitude': -35.6469,
              'longitude': 142.376,
              'surface_tempera

{'hotspots': {'confidence': 97,
              'latitude': -37.4885,
              'longitude': 142.0658,
              'surface_temperature_celsius': 81}}
{'hotspots': {'confidence': 77,
              'latitude': -36.7101,
              'longitude': 142.981,
              'surface_temperature_celsius': 68}}
{'hotspots': {'confidence': 100,
              'latitude': -37.9594,
              'longitude': 143.5638,
              'surface_temperature_celsius': 87}}
{'hotspots': {'confidence': 98,
              'latitude': -36.5921,
              'longitude': 144.4905,
              'surface_temperature_celsius': 82}}
{'hotspots': {'confidence': 96,
              'latitude': -36.6787,
              'longitude': 144.8042,
              'surface_temperature_celsius': 79}}
{'hotspots': {'confidence': 84,
              'latitude': -38.0423,
              'longitude': 146.4048,
              'surface_temperature_celsius': 66}}
{'hotspots': {'confidence': 94,
              'latitude': -38.1224,
  

{'hotspots': {'confidence': 91,
              'latitude': -36.4168,
              'longitude': 141.6939,
              'surface_temperature_celsius': 68}}
{'hotspots': {'confidence': 97,
              'latitude': -36.4211,
              'longitude': 141.6875,
              'surface_temperature_celsius': 81}}
{'hotspots': {'confidence': 96,
              'latitude': -36.8094,
              'longitude': 142.8885,
              'surface_temperature_celsius': 77}}
{'hotspots': {'confidence': 84,
              'latitude': -37.8323,
              'longitude': 147.2232,
              'surface_temperature_celsius': 68}}
{'hotspots': {'confidence': 98,
              'latitude': -38.2853,
              'longitude': 145.9519,
              'surface_temperature_celsius': 83}}
{'hotspots': {'confidence': 98,
              'latitude': -38.2375,
              'longitude': 142.8363,
              'surface_temperature_celsius': 82}}
{'hotspots': {'confidence': 92,
              'latitude': -37.4463,
  

{'hotspots': {'confidence': 100,
              'latitude': -37.606,
              'longitude': 149.312,
              'surface_temperature_celsius': 87}}
{'hotspots': {'confidence': 97,
              'latitude': -37.61,
              'longitude': 149.307,
              'surface_temperature_celsius': 80}}
{'hotspots': {'confidence': 99,
              'latitude': -37.609,
              'longitude': 149.32,
              'surface_temperature_celsius': 84}}
{'hotspots': {'confidence': 100,
              'latitude': -37.634,
              'longitude': 149.237,
              'surface_temperature_celsius': 71}}
{'hotspots': {'confidence': 98,
              'latitude': -37.605,
              'longitude': 149.302,
              'surface_temperature_celsius': 83}}
{'hotspots': {'confidence': 99,
              'latitude': -37.6,
              'longitude': 149.325,
              'surface_temperature_celsius': 86}}
{'hotspots': {'confidence': 95,
              'latitude': -37.618,
              'lo

In [11]:
# B.2c - Find date, surface_temp, air_temp, relative_humidity and max_wind_speed on 15,16 of Dec 2017
results = climates.find({"_id":{"$in":[datetime(2017,12,15,0,0),datetime(2017,12,16,0,0)]}},\
                        {"_id":1,"hotspots.surface_temperature_celsius":1,"air_temperature_celsius":1,\
                         "relative_humidity":1,"max_wind_speed":1})
for result in results:
    pprint(result)

{'_id': datetime.datetime(2017, 12, 15, 0, 0),
 'air_temperature_celsius': 18,
 'hotspots': [{'surface_temperature_celsius': 42},
              {'surface_temperature_celsius': 36},
              {'surface_temperature_celsius': 38},
              {'surface_temperature_celsius': 40}],
 'max_wind_speed': 14.0,
 'relative_humidity': 52.0}
{'_id': datetime.datetime(2017, 12, 16, 0, 0),
 'air_temperature_celsius': 18,
 'hotspots': [{'surface_temperature_celsius': 43},
              {'surface_temperature_celsius': 33},
              {'surface_temperature_celsius': 54},
              {'surface_temperature_celsius': 73},
              {'surface_temperature_celsius': 55},
              {'surface_temperature_celsius': 75},
              {'surface_temperature_celsius': 55},
              {'surface_temperature_celsius': 66},
              {'surface_temperature_celsius': 56},
              {'surface_temperature_celsius': 60},
              {'surface_temperature_celsius': 73},
              {'surface

In [12]:
# B.2d - Find datetime, surface_temp, air_temp, confidence when confidence is between 80 and 100
results = climates.aggregate([{"$unwind":"$hotspots"},\
                              {"$match":{"hotspots.confidence":{"$gt":80,"$lt":100}}},\
                              {"$project":{"_id":0,"hotspots.datetime":1,"air_temperature_celsius":1,\
                                           "hotspots.surface_temperature_celsius":1,"hotspots.confidence":1}}])
for result in results:
    pprint(result)

{'air_temperature_celsius': 20,
 'hotspots': {'confidence': 87,
              'datetime': datetime.datetime(2017, 3, 6, 5, 6, 30),
              'surface_temperature_celsius': 62}}
{'air_temperature_celsius': 20,
 'hotspots': {'confidence': 85,
              'datetime': datetime.datetime(2017, 3, 6, 5, 6, 20),
              'surface_temperature_celsius': 59}}
{'air_temperature_celsius': 19,
 'hotspots': {'confidence': 88,
              'datetime': datetime.datetime(2017, 3, 7, 4, 16, 10),
              'surface_temperature_celsius': 64}}
{'air_temperature_celsius': 23,
 'hotspots': {'confidence': 86,
              'datetime': datetime.datetime(2017, 3, 9, 13, 23, 40),
              'surface_temperature_celsius': 41}}
{'air_temperature_celsius': 19,
 'hotspots': {'confidence': 94,
              'datetime': datetime.datetime(2017, 3, 10, 4, 45, 30),
              'surface_temperature_celsius': 73}}
{'air_temperature_celsius': 19,
 'hotspots': {'confidence': 97,
              'datetime': 

{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 93,
              'datetime': datetime.datetime(2017, 4, 4, 15, 31),
              'surface_temperature_celsius': 44}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 88,
              'datetime': datetime.datetime(2017, 4, 4, 4, 41, 50),
              'surface_temperature_celsius': 63}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 82,
              'datetime': datetime.datetime(2017, 4, 4, 4, 41, 10),
              'surface_temperature_celsius': 62}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 88,
              'datetime': datetime.datetime(2017, 4, 4, 4, 41),
              'surface_temperature_celsius': 64}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 86,
              'datetime': datetime.datetime(2017, 4, 4, 4, 41),
              'surface_temperature_celsius': 61}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 90,
              'datetime': datetime.da

{'air_temperature_celsius': 19,
 'hotspots': {'confidence': 91,
              'datetime': datetime.datetime(2017, 4, 6, 4, 22, 20),
              'surface_temperature_celsius': 69}}
{'air_temperature_celsius': 19,
 'hotspots': {'confidence': 86,
              'datetime': datetime.datetime(2017, 4, 6, 4, 22, 10),
              'surface_temperature_celsius': 68}}
{'air_temperature_celsius': 19,
 'hotspots': {'confidence': 85,
              'datetime': datetime.datetime(2017, 4, 6, 4, 22, 10),
              'surface_temperature_celsius': 59}}
{'air_temperature_celsius': 19,
 'hotspots': {'confidence': 90,
              'datetime': datetime.datetime(2017, 4, 6, 4, 22, 10),
              'surface_temperature_celsius': 67}}
{'air_temperature_celsius': 19,
 'hotspots': {'confidence': 90,
              'datetime': datetime.datetime(2017, 4, 6, 4, 21, 30),
              'surface_temperature_celsius': 66}}
{'air_temperature_celsius': 19,
 'hotspots': {'confidence': 94,
              'datetime': 

              'surface_temperature_celsius': 60}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 92,
              'datetime': datetime.datetime(2017, 4, 13, 4, 36, 30),
              'surface_temperature_celsius': 70}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 89,
              'datetime': datetime.datetime(2017, 4, 13, 4, 36, 20),
              'surface_temperature_celsius': 65}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 89,
              'datetime': datetime.datetime(2017, 4, 13, 4, 35, 40),
              'surface_temperature_celsius': 64}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 85,
              'datetime': datetime.datetime(2017, 4, 13, 4, 35, 40),
              'surface_temperature_celsius': 59}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 86,
              'datetime': datetime.datetime(2017, 4, 13, 4, 35, 30),
              'surface_temperature_celsius': 60}}
{'air_temperature_celsius': 16,
 'h

              'datetime': datetime.datetime(2017, 4, 13, 4, 26, 30),
              'surface_temperature_celsius': 60}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 88,
              'datetime': datetime.datetime(2017, 4, 13, 4, 26, 30),
              'surface_temperature_celsius': 63}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 89,
              'datetime': datetime.datetime(2017, 4, 13, 4, 26, 30),
              'surface_temperature_celsius': 65}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 84,
              'datetime': datetime.datetime(2017, 4, 13, 4, 26, 30),
              'surface_temperature_celsius': 58}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 99,
              'datetime': datetime.datetime(2017, 4, 13, 4, 26, 30),
              'surface_temperature_celsius': 85}}
{'air_temperature_celsius': 16,
 'hotspots': {'confidence': 83,
              'datetime': datetime.datetime(2017, 4, 13, 4, 26, 30),
              's

...

              'surface_temperature_celsius': 58}}
{'air_temperature_celsius': 15,
 'hotspots': {'confidence': 91,
              'datetime': datetime.datetime(2017, 4, 18, 4, 44, 50),
              'surface_temperature_celsius': 68}}
{'air_temperature_celsius': 15,
 'hotspots': {'confidence': 94,
              'datetime': datetime.datetime(2017, 4, 18, 4, 44, 50),
              'surface_temperature_celsius': 73}}
{'air_temperature_celsius': 15,
 'hotspots': {'confidence': 82,
              'datetime': datetime.datetime(2017, 4, 18, 4, 44, 50),
              'surface_temperature_celsius': 56}}
{'air_temperature_celsius': 15,
 'hotspots': {'confidence': 86,
              'datetime': datetime.datetime(2017, 4, 18, 4, 44, 50),
              'surface_temperature_celsius': 61}}
{'air_temperature_celsius': 15,
 'hotspots': {'confidence': 92,
              'datetime': datetime.datetime(2017, 4, 18, 4, 44, 50),
              'surface_temperature_celsius': 70}}
{'air_temperature_celsius': 15,
 'h

              'surface_temperature_celsius': 64}}
{'air_temperature_celsius': 10,
 'hotspots': {'confidence': 84,
              'datetime': datetime.datetime(2017, 5, 3, 4, 2, 20),
              'surface_temperature_celsius': 58}}
{'air_temperature_celsius': 10,
 'hotspots': {'confidence': 93,
              'datetime': datetime.datetime(2017, 5, 3, 4, 2, 20),
              'surface_temperature_celsius': 73}}
{'air_temperature_celsius': 10,
 'hotspots': {'confidence': 85,
              'datetime': datetime.datetime(2017, 5, 3, 4, 2, 20),
              'surface_temperature_celsius': 60}}
{'air_temperature_celsius': 10,
 'hotspots': {'confidence': 90,
              'datetime': datetime.datetime(2017, 5, 3, 4, 2, 20),
              'surface_temperature_celsius': 67}}
{'air_temperature_celsius': 10,
 'hotspots': {'confidence': 84,
              'datetime': datetime.datetime(2017, 5, 3, 4, 2, 20),
              'surface_temperature_celsius': 58}}
{'air_temperature_celsius': 10,
 'hotspots': 

              'datetime': datetime.datetime(2017, 5, 10, 4, 8, 30),
              'surface_temperature_celsius': 61}}
{'air_temperature_celsius': 10,
 'hotspots': {'confidence': 83,
              'datetime': datetime.datetime(2017, 5, 10, 4, 8, 30),
              'surface_temperature_celsius': 56}}
{'air_temperature_celsius': 10,
 'hotspots': {'confidence': 95,
              'datetime': datetime.datetime(2017, 5, 10, 4, 8, 10),
              'surface_temperature_celsius': 76}}
{'air_temperature_celsius': 10,
 'hotspots': {'confidence': 85,
              'datetime': datetime.datetime(2017, 5, 10, 4, 8, 10),
              'surface_temperature_celsius': 59}}
{'air_temperature_celsius': 10,
 'hotspots': {'confidence': 99,
              'datetime': datetime.datetime(2017, 5, 10, 4, 8, 10),
              'surface_temperature_celsius': 84}}
{'air_temperature_celsius': 10,
 'hotspots': {'confidence': 96,
              'datetime': datetime.datetime(2017, 5, 10, 4, 8, 10),
              'surface

 'hotspots': {'confidence': 85,
              'datetime': datetime.datetime(2017, 5, 26, 4, 8, 10),
              'surface_temperature_celsius': 59}}
{'air_temperature_celsius': 11,
 'hotspots': {'confidence': 82,
              'datetime': datetime.datetime(2017, 6, 1, 5, 9, 20),
              'surface_temperature_celsius': 56}}
{'air_temperature_celsius': 12,
 'hotspots': {'confidence': 83,
              'datetime': datetime.datetime(2017, 6, 2, 4, 14, 10),
              'surface_temperature_celsius': 57}}
{'air_temperature_celsius': 12,
 'hotspots': {'confidence': 83,
              'datetime': datetime.datetime(2017, 6, 2, 4, 14, 10),
              'surface_temperature_celsius': 57}}
{'air_temperature_celsius': 9,
 'hotspots': {'confidence': 86,
              'datetime': datetime.datetime(2017, 6, 4, 4, 8, 20),
              'surface_temperature_celsius': 61}}
{'air_temperature_celsius': 9,
 'hotspots': {'confidence': 91,
              'datetime': datetime.datetime(2017, 6, 4, 4, 8, 

 'hotspots': {'confidence': 88,
              'datetime': datetime.datetime(2017, 11, 22, 3, 46),
              'surface_temperature_celsius': 64}}
{'air_temperature_celsius': 24,
 'hotspots': {'confidence': 84,
              'datetime': datetime.datetime(2017, 11, 23, 4, 28, 59),
              'surface_temperature_celsius': 60}}
{'air_temperature_celsius': 24,
 'hotspots': {'confidence': 93,
              'datetime': datetime.datetime(2017, 11, 23, 4, 28, 59),
              'surface_temperature_celsius': 73}}
{'air_temperature_celsius': 28,
 'hotspots': {'confidence': 97,
              'datetime': datetime.datetime(2017, 11, 29, 3, 52, 10),
              'surface_temperature_celsius': 80}}
{'air_temperature_celsius': 28,
 'hotspots': {'confidence': 99,
              'datetime': datetime.datetime(2017, 11, 29, 3, 52, 10),
              'surface_temperature_celsius': 84}}
{'air_temperature_celsius': 26,
 'hotspots': {'confidence': 84,
              'datetime': datetime.datetime(2017, 11

In [13]:
# B.2e - Find the top 10 records with highest surface temperature
results = climates.aggregate([{"$unwind":"$hotspots"},\
                              {"$project":{"_id":1,"max_wind_speed":1,"air_temperature_celsius":1,"max_wind_speed":1\
                                           ,"precipitation":1,"relative_humidity":1,"station":1,"windspeed_knots":1\
                                           ,"hotspots.datetime":1,"hotspots.latitude":1,"hotspots.longitude":1\
                                           ,"hotspots.surface_temperature_celsius":1,"hotspots.confidence":1}},
                             {"$sort":{"hotspots.surface_temperature_celsius":-1}},
                             {"$limit":10}])
for result in results:
    pprint(result)

{'_id': datetime.datetime(2017, 4, 18, 0, 0),
 'air_temperature_celsius': 15,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2017, 4, 18, 4, 52),
              'latitude': -38.1665,
              'longitude': 143.062,
              'surface_temperature_celsius': 124},
 'max_wind_speed': 9.9,
 'precipitation': ' 0.00I',
 'relative_humidity': 56.1,
 'station': 948701,
 'windspeed_knots': 5.1}
{'_id': datetime.datetime(2017, 4, 4, 0, 0),
 'air_temperature_celsius': 16,
 'hotspots': {'confidence': 100,
              'datetime': datetime.datetime(2017, 4, 4, 4, 32, 50),
              'latitude': -36.343,
              'longitude': 142.1986,
              'surface_temperature_celsius': 123},
 'max_wind_speed': 12.0,
 'precipitation': ' 0.00I',
 'relative_humidity': 47.5,
 'station': 948701,
 'windspeed_knots': 5.4}
{'_id': datetime.datetime(2017, 5, 1, 0, 0),
 'air_temperature_celsius': 14,
 'hotspots': {'confidence': 100,
              'datetime': datetime.date

In [16]:
# B.2f - Find the number of fire in each day
results = climates.aggregate([{"$unwind":"$hotspots"},{"$group":{"_id":"$_id","number of fires":{"$sum":1}}},\
                             {"$sort":{"_id":1}}])
                              
for result in results:
    pprint(result)

{'_id': datetime.datetime(2017, 3, 6, 0, 0), 'number of fires': 2}
{'_id': datetime.datetime(2017, 3, 7, 0, 0), 'number of fires': 1}
{'_id': datetime.datetime(2017, 3, 8, 0, 0), 'number of fires': 2}
{'_id': datetime.datetime(2017, 3, 9, 0, 0), 'number of fires': 3}
{'_id': datetime.datetime(2017, 3, 10, 0, 0), 'number of fires': 8}
{'_id': datetime.datetime(2017, 3, 12, 0, 0), 'number of fires': 5}
{'_id': datetime.datetime(2017, 3, 13, 0, 0), 'number of fires': 2}
{'_id': datetime.datetime(2017, 3, 14, 0, 0), 'number of fires': 10}
{'_id': datetime.datetime(2017, 3, 15, 0, 0), 'number of fires': 7}
{'_id': datetime.datetime(2017, 3, 17, 0, 0), 'number of fires': 6}
{'_id': datetime.datetime(2017, 3, 18, 0, 0), 'number of fires': 3}
{'_id': datetime.datetime(2017, 3, 19, 0, 0), 'number of fires': 21}
{'_id': datetime.datetime(2017, 3, 24, 0, 0), 'number of fires': 2}
{'_id': datetime.datetime(2017, 3, 25, 0, 0), 'number of fires': 13}
{'_id': datetime.datetime(2017, 3, 26, 0, 0), 'nu

In [19]:
# B.2g - Find average surface temperature for each day
results = climates.aggregate([{"$unwind":"$hotspots"},\
                              {"$group":{"_id":"$_id","avg surface temp":\
                                         {"$avg":"$hotspots.surface_temperature_celsius"}}},\
                                         {"$sort":{"_id":1}}])
                              
for result in results:
    pprint(result)

{'_id': datetime.datetime(2017, 3, 6, 0, 0), 'avg surface temp': 60.5}
{'_id': datetime.datetime(2017, 3, 7, 0, 0), 'avg surface temp': 64.0}
{'_id': datetime.datetime(2017, 3, 8, 0, 0), 'avg surface temp': 51.5}
{'_id': datetime.datetime(2017, 3, 9, 0, 0),
 'avg surface temp': 46.666666666666664}
{'_id': datetime.datetime(2017, 3, 10, 0, 0), 'avg surface temp': 69.375}
{'_id': datetime.datetime(2017, 3, 12, 0, 0), 'avg surface temp': 88.2}
{'_id': datetime.datetime(2017, 3, 13, 0, 0), 'avg surface temp': 38.5}
{'_id': datetime.datetime(2017, 3, 14, 0, 0), 'avg surface temp': 65.6}
{'_id': datetime.datetime(2017, 3, 15, 0, 0), 'avg surface temp': 46.0}
{'_id': datetime.datetime(2017, 3, 17, 0, 0), 'avg surface temp': 59.5}
{'_id': datetime.datetime(2017, 3, 18, 0, 0),
 'avg surface temp': 79.33333333333333}
{'_id': datetime.datetime(2017, 3, 19, 0, 0),
 'avg surface temp': 65.57142857142857}
{'_id': datetime.datetime(2017, 3, 24, 0, 0), 'avg surface temp': 49.0}
{'_id': datetime.dateti